In [1]:
import sklearn
import numpy as np
from sklearn import preprocessing
import h5py
from sklearn.metrics import mean_squared_error
import pandas as pd
import math
from matplotlib import pyplot as plt
fname = 'BJ{}_M32x32_T30_InOut.h5'.format(16)
reader=h5py.File(fname,'r')
data=reader['data'].value
date=reader['date'].value

In [2]:
train_data_all=[]
for timeslot in data:
    train_data_all.append(timeslot[0][15][10])

In [3]:
train_x=[]
train_y=[]
lookback=48
for i in range(len(train_data_all)-lookback):
    a=train_data_all[i:(i+lookback)]
    train_x.append(a)
    train_y.append(train_data_all[i+lookback])
print(len(train_x))

7172


In [4]:
minmaxx=preprocessing.MinMaxScaler()
minmaxy=preprocessing.MinMaxScaler()
train_x=minmaxx.fit_transform(train_x)
train_y=np.array(train_y)
train_y=train_y.reshape(-1,1)
train_y=minmaxy.fit_transform(train_y)

In [5]:
train_size=1000
train_xt=train_x[:train_size]
train_yt=train_y[:train_size]
import torch
device = torch.device("cuda")
train_xt = np.array(train_xt).reshape(-1, 1, lookback)
train_yt = np.array(train_yt).reshape(-1, 1, 1)

train_xt = torch.from_numpy(train_xt).to(device)
train_yt = torch.from_numpy(train_yt).to(device)

In [6]:
from torch import nn
import torch
from torch.autograd import Variable
from torch import  tensor
# 定义模型

class lstm_reg(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super(lstm_reg, self).__init__()
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers) # rnn
        self.reg = nn.Linear(hidden_size, output_size) # 回归
        
    def forward(self, x):
        x, _ = self.rnn(x) # (seq, batch, hidden)
        s, b, h = x.shape
        x = x.view(s*b, h) # 转换成线性层的输入格式
        x = self.reg(x)
        x = x.view(s, b, -1)
        return x
net = lstm_reg(48, 20)
net.double()
net.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
# 开始训练
for e in range(1000):
    var_x = tensor(train_xt)
    var_y = tensor(train_yt)
    # 前向传播
    out = net(var_x)
    loss = criterion(out, var_y)
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (e + 1) % 100 == 0: # 每 100 次输出结果
        print('Epoch: {}, Loss: {:.10f}'.format(e + 1, loss.data[0]))



C:\Users\hu\Miniconda3\envs\pytorch-gpu\lib\site-packages\ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: 100, Loss: 0.0092202064
Epoch: 200, Loss: 0.0059458346
Epoch: 300, Loss: 0.0055417457
Epoch: 400, Loss: 0.0047681977
Epoch: 500, Loss: 0.0037128043
Epoch: 600, Loss: 0.0031984655
Epoch: 700, Loss: 0.0023596369
Epoch: 800, Loss: 0.0016147790
Epoch: 900, Loss: 0.0012592709
Epoch: 1000, Loss: 0.0007989387


In [10]:
net.eval()

lstm_reg(
  (rnn): LSTM(48, 20, num_layers=2)
  (reg): Linear(in_features=20, out_features=1, bias=True)
)

In [17]:
out=net(train_xt[0:500])

In [18]:
y=train_yt[0:500]

In [30]:
s=0
for i in range(1000):
    pre=minmaxy.inverse_transform(out[i].float().data)
    t=minmaxy.inverse_transform(train_yt[i].float().data)
    s+=mean_squared_error(pre,t)

IndexError: index 500 is out of bounds for dimension 0 with size 500

In [31]:
print(s)

42599.15814743112


array([[252.9999975]])